In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import datetime

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SHARED_STORE, SNOBAL_DIR, SNOTEL_DIR, DATA_DIR, COARSEN_OPTS, HV_PLOT_OPTS

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

In [ ]:
water_year = f'wy2021'

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

In [ ]:
# NOTE: MesoWest data is in delivered as UTC time
PD_OPTS = dict(
    parse_dates=True,
    index_col=0,
    names=['Depth', 'Air-T', '-', 'RH', 'Wind', '--'],
    header=0,
)

irwin_guide = pd.concat([
    pd.read_csv(DATA_DIR / 'Snotel/wy2021/Irwin/usda-csv/2021-Irwin.csv', **PD_OPTS),
    pd.read_csv(DATA_DIR / 'Snotel/wy2022/Irwin/usda-csv/2022-Irwin.csv', **PD_OPTS),
])


## Model run with HRRR clouds and solar + MODIS albedo

In [ ]:
HRRR_modis_21 = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/wy2021/erw_hrrr_solar_modis_cubic/run*/smrf_*.nc',
    parallel=True, chunks={'time': 24},
    drop_variables=['percent_snow', 'precip_temp', 'precip', 'snow_density', 'storm_days', 'thermal', 'vapor_pressure'],
).sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

In [ ]:
HRRR_modis_22 = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/wy2022/erw_hrrr_solar_modis_cubic/run*/smrf_*.nc',
    parallel=True, chunks={'time': 24},
    drop_variables=['percent_snow', 'precip_temp', 'precip', 'snow_density', 'storm_days', 'thermal', 'vapor_pressure'],
).sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

In [ ]:
HRRR_modis = xr.merge([HRRR_modis_21, HRRR_modis_22])

## Model Bias

In [ ]:
def in_date_range_xr(data, start, end):
    return data.where(
        data.time >= start
    ).where(
        data.time <= end
    )

def rolling_in_date_range_pd(df, start, end):
    return df[
        (df.index >= start) & (df.index < end)
    ].rolling(24).mean()

In [ ]:
HRRR_pd = HRRR_modis.squeeze(['x', 'y']).to_dataframe().drop(['x', 'y', 'projection'], axis=1)

irwin_guide.index = irwin_guide.index.tz_localize(None)

In [ ]:
model_bias_air_t = HRRR_pd['air_temp'] - irwin_guide['Air-T']
model_bias_wind = HRRR_pd['wind_speed'] - irwin_guide['Wind']

## Matplotlib 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from matplotlib.offsetbox import AnchoredText

In [ ]:
pd.options.plotting.backend = 'matplotlib'

In [ ]:
def model_axes(axes, x_pos=1.05):    
    model_cax = axes.twinx()
    model_cax.set_ylabel(r'Model bias', rotation=270, labelpad=4, ha='center', fontsize=10, fontstyle='italic')
    model_cax.set_yticks([0,1])
    model_cax.set_yticklabels([])
    
    label_style=dict(
        xycoords='axes fraction',
        va="center", 
        ha="center",
        fontsize=10,
        rotation=270,
    )
    
    model_cax.annotate("Over",
        xy=(x_pos, 1),
        xytext=(x_pos, .84),
        arrowprops=dict(arrowstyle="simple", fc='blue'),
        **label_style
    )
    model_cax.annotate("Under",
        xy=(x_pos, 0.00),
        xytext=(x_pos, .18),
        arrowprops=dict(arrowstyle="simple", fc='red'),
        **label_style
    )

In [ ]:
figure_opts = dict(figsize=(7,6), dpi=300,)
fig, axes = plt.subplots(2, 1, sharex=True, **figure_opts)
plt.subplots_adjust(hspace=0.05)

for ax in axes:
    ax.axhline(0, lw=0.8, ls=(1, (1, 2)), color='black', zorder=0, alpha=0.75)

LINE_OPTS_2021 = dict(label='2021', color='steelblue', alpha=0.8, lw=1)
LINE_OPTS_2022 = dict(label='2022', color='peru', alpha=0.8, lw=1.2, ls=(0, (5, 1)))

# Air T
## 2021
data = rolling_in_date_range_pd(model_bias_air_t, '2020-10-01', '2021-06-30')
print('Air-T 2021')
print(data.describe())

axes[0].plot(
    data.index.shift(freq = pd.DateOffset(years = 1)), data.values, 
    **LINE_OPTS_2021,
)

## 2022
data = rolling_in_date_range_pd(model_bias_air_t, '2021-10-01', '2022-06-30')

print('Air-T 2022')
print(data.describe())

axes[0].plot(
    data.index, data.values, 
    **LINE_OPTS_2022,
)
axes[0].set_ylabel('Δ Air Temperature (°C)')

# Wind
## 2021
data = rolling_in_date_range_pd(model_bias_wind, '2020-10-01', '2021-06-30')

print('Wind 2021')
print(data.describe())

axes[1].plot(
    data.index.shift(freq = pd.DateOffset(years = 1)), data.values, 
    **LINE_OPTS_2021,
)

## 2022
data = rolling_in_date_range_pd(model_bias_wind, '2021-10-01', '2022-06-30')

print('Wind 2022')
print(data.describe())

axes[1].plot(
    data.index, data.values, 
    **LINE_OPTS_2022,
)
axes[1].set_ylabel('Δ Wind (m/s)')

# Date axis
axes[1].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
axes[1].xaxis.set_minor_locator(mticker.NullLocator())
axes[1].set_xlim(np.datetime64('2021-10-01'), np.datetime64('2022-07-01'))

for ax in axes:
    model_axes(ax, 1.015)
    ax.legend(loc='lower right', frameon=False, borderaxespad=0.15, fontsize=8);

In [ ]:
import hvplot.xarray
import hvplot.pandas
import holoviews as hv

pd.options.plotting.backend = 'holoviews'

In [ ]:
rolling_in_date_range_pd(model_bias_air_t, '2020-10-01', '2021-06-30').plot(label='wy2021', **HV_PLOT_OPTS, title='Air Temperature') * \
rolling_in_date_range_pd(model_bias_air_t, '2021-10-01', '2022-06-30').shift(-365, freq='D').plot(label='wy2022', **HV_PLOT_OPTS) * \
hv.HLine(0).opts(color='grey', line_width=2, line_dash='dotted')

In [ ]:
rolling_in_date_range_pd(model_bias_wind, '2020-10-01', '2021-06-30').plot(label='wy2021', **HV_PLOT_OPTS, title='Wind Speed') * \
rolling_in_date_range_pd(model_bias_wind, '2021-10-01', '2022-06-30').shift(-365, freq='D').plot(label='wy2022', **HV_PLOT_OPTS) * \
hv.HLine(0).opts(color='grey', line_width=2, line_dash='dotted')

In [ ]:
client.shutdown()